# import basic

In [1]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests

In [2]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# list of object of animals

animals = {
      "dog": {
        "sound": "bark",
      },
      "cat": {
        "sound": "meow",
      },
      "bird": {
        "sound": "tweet",
      },
      "fish": {
        "sound": "bubble",
      },
      "snake": {
        "sound": "hiss",
      }
    }

def search_animal_secret_sound(animal_name):
    return animals[animal_name]["sound"]

In [4]:
search_animal_json = {
    "name": "search_animal_secret_sound_function",
    "description": "Use this tool to search for an animal by name",
    "parameters": {
        "type": "object",
        "properties": {
            "animal_name": {
                "type": "string",
                "description": "The name of the animal to search for"
            }
        },
        "required": ["animal_name"],
        "additionalProperties": False
    }
}

In [5]:
tools = [{"type": "function", "function": search_animal_json}]

In [ ]:
tools

In [7]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: \n {tool_name}", flush=True)
        if tool_name == "search_animal_secret_sound_function":
            result = search_animal_secret_sound(**arguments)
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
message="i want to know the secret sound of a dog"
messages = [{"role": "system", "content": ""}]  + [{"role": "user", "content": message}]

bStream = False
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

finish_reason = response.choices[0].finish_reason

if finish_reason=="tool_calls":
    message = response.choices[0].message
    tool_calls = message.tool_calls
    # print(tool_calls)
    results = handle_tool_calls(tool_calls)
    print('\nTool call Results: \n', results)
    messages.append(message)
    messages.extend(results)

    bStream = True
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True  # 스트리밍 옵션 추가!
    )

# print result
print('\nFinal Response:');
if bStream == False:
    print(response.choices[0].message.content)
else:
    for chunk in response:
        # 각 chunk에는 부분 응답이 들어있음
        print(chunk.choices[0].delta.content, end="", flush=True)

